In [1]:
# %% [markdown]
# # 绘制训练结果图表
# ## 包含：网格搜索热力图 + 数据量曲线 + 组分分析

# %% 导入库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 设置中文字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
matplotlib.rcParams['axes.unicode_minus'] = False

print("✅ 库导入成功")

# %% 配置路径
OUT_DIR = r"F:\model_outputs_2020_resnet_optimized"
GRID_CSV = os.path.join(OUT_DIR, "stage1_grid_search.csv")
FRACTION_CSV = os.path.join(OUT_DIR, "stage2_fraction_results.csv")

# 检查文件是否存在
if not os.path.exists(GRID_CSV):
    print(f"⚠️ 找不到阶段1结果: {GRID_CSV}")
if not os.path.exists(FRACTION_CSV):
    print(f"⚠️ 找不到阶段2结果: {FRACTION_CSV}")

# %% [markdown]
# ## 图1：网格搜索热力图

# %% 绘制网格搜索热力图
def plot_grid_search_heatmap():
    """阶段1：网格搜索结果"""
    if not os.path.exists(GRID_CSV):
        print("⚠️ 跳过网格搜索图")
        return
    
    df = pd.read_csv(GRID_CSV)
    if df.empty:
        print("⚠️ 数据为空")
        return
    
    # 过滤错误
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    # 计算每个超参组合的平均R²
    pivot = df.groupby(['lr', 'weight_decay'])['test_r2_mean'].mean().reset_index()
    pivot_table = pivot.pivot(index='weight_decay', columns='lr', values='test_r2_mean')
    
    fig, ax = plt.subplots(figsize=(10, 7))
    im = ax.imshow(pivot_table.values, cmap='YlGnBu', aspect='auto', 
                   vmin=0, vmax=1, interpolation='nearest')
    
    # 设置刻度
    ax.set_xticks(range(len(pivot_table.columns)))
    ax.set_xticklabels([f"{x:.0e}" for x in pivot_table.columns], rotation=45)
    ax.set_yticks(range(len(pivot_table.index)))
    ax.set_yticklabels([f"{x:.0e}" for x in pivot_table.index])
    
    # 标注数值
    for i in range(len(pivot_table.index)):
        for j in range(len(pivot_table.columns)):
            val = pivot_table.values[i, j]
            if not np.isnan(val):
                text_color = 'white' if val < 0.5 else 'black'
                ax.text(j, i, f'{val:.3f}', ha='center', va='center', 
                       color=text_color, fontsize=11, weight='bold')
    
    ax.set_xlabel('Learning Rate', fontsize=13, weight='bold')
    ax.set_ylabel('Weight Decay', fontsize=13, weight='bold')
    ax.set_title('阶段1：超参数网格搜索 - 平均测试R²', fontsize=15, weight='bold', pad=20)
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Mean Test R²', fontsize=12, weight='bold')
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig1_grid_search_heatmap.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 已输出: fig1_grid_search_heatmap.png")

plot_grid_search_heatmap()

# %% [markdown]
# ## 图2：数据量 vs 性能曲线

# %% 绘制数据量曲线
def plot_fraction_vs_performance():
    """阶段2：训练数据量对性能的影响"""
    if not os.path.exists(FRACTION_CSV):
        print("⚠️ 跳过数据量曲线图")
        return
    
    df = pd.read_csv(FRACTION_CSV)
    if df.empty:
        return
    
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    # 计算每个fraction的统计量
    grouped = df.groupby('fraction')['test_r2_mean'].agg(['mean', 'std', 'count']).reset_index()
    
    fig, ax = plt.subplots(figsize=(11, 7))
    
    # 绘制误差线
    ax.errorbar(
        grouped['fraction'] * 100,
        grouped['mean'],
        yerr=grouped['std'].fillna(0),
        marker='o',
        markersize=10,
        capsize=6,
        capthick=2,
        linewidth=2.5,
        color='#2E86AB',
        ecolor='#A23B72',
        label='平均R² ± 标准差'
    )
    
    # 标注数据点
    for _, row in grouped.iterrows():
        ax.annotate(
            f"{row['mean']:.3f}\n(n={int(row['count'])})",
            xy=(row['fraction']*100, row['mean']),
            xytext=(0, 15),
            textcoords='offset points',
            ha='center',
            fontsize=9,
            bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3)
        )
    
    ax.set_xlabel('训练数据比例 (%)', fontsize=13, weight='bold')
    ax.set_ylabel('测试集平均R²', fontsize=13, weight='bold')
    ax.set_title('阶段2：训练数据量 vs 模型性能', fontsize=15, weight='bold', pad=20)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(fontsize=11, loc='lower right')
    ax.set_ylim(bottom=0)
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig2_fraction_vs_r2.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📈 已输出: fig2_fraction_vs_r2.png")

plot_fraction_vs_performance()

# %% [markdown]
# ## 图3：各组分R²箱线图

# %% 绘制组分箱线图
def plot_component_boxplots():
    """100%数据时各组分的R²分布"""
    if not os.path.exists(FRACTION_CSV):
        print("⚠️ 跳过组分箱线图")
        return
    
    df = pd.read_csv(FRACTION_CSV)
    if df.empty:
        return
    
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    # 只看fraction=1.0的数据
    df_full = df[np.isclose(df["fraction"], 1.0)]
    if df_full.empty:
        print("⚠️ 没有100%数据")
        return
    
    # 找出组分列
    target_cols = [c for c in df_full.columns 
                   if c.startswith("test_r2_") and c != "test_r2_mean"]
    if not target_cols:
        return
    
    data = [df_full[col].dropna().values for col in target_cols]
    labels = [c.replace("test_r2_", "") for c in target_cols]
    
    fig, ax = plt.subplots(figsize=(11, 7))
    
    bp = ax.boxplot(
        data,
        labels=labels,
        patch_artist=True,
        notch=True,
        showmeans=True,
        meanprops=dict(marker='D', markerfacecolor='red', markersize=8, 
                      markeredgecolor='darkred', linewidth=1.5),
        medianprops=dict(color='darkblue', linewidth=2),
        whiskerprops=dict(linewidth=1.5),
        capprops=dict(linewidth=1.5)
    )
    
    # 设置颜色
    colors = ['#A8DADC', '#457B9D', '#F1FAEE', '#E63946']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
        patch.set_edgecolor('black')
        patch.set_linewidth(1.5)
    
    ax.set_ylabel('R²', fontsize=13, weight='bold')
    ax.set_xlabel('组分', fontsize=13, weight='bold')
    ax.set_title('各组分R²分布（100%训练数据）', fontsize=15, weight='bold', pad=20)
    ax.grid(True, axis='y', alpha=0.3, linestyle='--')
    
    # 添加参考线
    ax.axhline(y=0.9, color='green', linestyle='--', linewidth=1.5, 
               alpha=0.6, label='R²=0.9')
    ax.axhline(y=0.8, color='orange', linestyle='--', linewidth=1.5, 
               alpha=0.6, label='R²=0.8')
    ax.legend(fontsize=10)
    
    # 标注统计信息
    for i, (d, label) in enumerate(zip(data, labels)):
        mean_val = np.mean(d)
        ax.text(i+1, ax.get_ylim()[1]*0.95, f'μ={mean_val:.3f}', 
               ha='center', fontsize=9, weight='bold')
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig3_component_boxplot.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📦 已输出: fig3_component_boxplot.png")

plot_component_boxplots()

# %% [markdown]
# ## 图4：折×组分热力图

# %% 绘制折×组分热力图
def plot_fold_component_heatmap():
    """各折在各组分上的表现"""
    if not os.path.exists(FRACTION_CSV):
        print("⚠️ 跳过热力图")
        return
    
    df = pd.read_csv(FRACTION_CSV)
    if df.empty:
        return
    
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    df_full = df[np.isclose(df["fraction"], 1.0)]
    if df_full.empty:
        return
    
    target_cols = [c for c in df_full.columns 
                   if c.startswith("test_r2_") and c != "test_r2_mean"]
    if not target_cols:
        return
    
    # 按fold分组计算平均
    heat_data = df_full.groupby('fold')[target_cols].mean()
    heat_data.columns = [c.replace("test_r2_", "") for c in heat_data.columns]
    
    fig, ax = plt.subplots(figsize=(10, 7))
    im = ax.imshow(heat_data.T, aspect='auto', vmin=0, vmax=1, 
                   cmap='RdYlGn', interpolation='nearest')
    
    ax.set_xticks(range(len(heat_data.index)))
    ax.set_xticklabels(heat_data.index, fontsize=11)
    ax.set_yticks(range(len(heat_data.columns)))
    ax.set_yticklabels(heat_data.columns, fontsize=11)
    
    # 标注数值
    for i in range(len(heat_data.columns)):
        for j in range(len(heat_data.index)):
            val = heat_data.iloc[j, i]
            if not np.isnan(val):
                text_color = 'white' if val < 0.5 else 'black'
                ax.text(j, i, f'{val:.2f}', ha='center', va='center',
                       color=text_color, fontsize=11, weight='bold')
    
    ax.set_xlabel('Fold', fontsize=13, weight='bold')
    ax.set_ylabel('组分', fontsize=13, weight='bold')
    ax.set_title('各折×各组分 R²热力图（100%数据）', fontsize=15, weight='bold', pad=20)
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('R²', fontsize=12, weight='bold')
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig4_fold_component_heatmap.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"🔥 已输出: fig4_fold_component_heatmap.png")

plot_fold_component_heatmap()

# %% [markdown]
# ## 图5：各组分学习曲线

# %% 绘制学习曲线
def plot_learning_curves():
    """不同数据量下各组分的学习曲线"""
    if not os.path.exists(FRACTION_CSV):
        print("⚠️ 跳过学习曲线")
        return
    
    df = pd.read_csv(FRACTION_CSV)
    if df.empty:
        return
    
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    target_cols = [c for c in df.columns 
                   if c.startswith("test_r2_") and c != "test_r2_mean"]
    if not target_cols:
        return
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    colors = ['#E63946', '#457B9D', '#2A9D8F', '#F4A261']
    markers = ['o', 's', '^', 'D']
    
    for idx, col in enumerate(target_cols):
        comp_name = col.replace("test_r2_", "")
        grouped = df.groupby('fraction')[col].agg(['mean', 'std']).reset_index()
        
        color = colors[idx % len(colors)]
        marker = markers[idx % len(markers)]
        
        ax.errorbar(
            grouped['fraction'] * 100,
            grouped['mean'],
            yerr=grouped['std'].fillna(0),
            marker=marker,
            markersize=8,
            capsize=5,
            linewidth=2,
            label=comp_name,
            color=color
        )
    
    ax.set_xlabel('训练数据比例 (%)', fontsize=13, weight='bold')
    ax.set_ylabel('组分R²', fontsize=13, weight='bold')
    ax.set_title('各组分性能 vs 训练数据量', fontsize=15, weight='bold', pad=20)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(fontsize=11, loc='lower right', framealpha=0.9)
    ax.set_ylim(bottom=0)
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig5_component_learning_curves.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📉 已输出: fig5_component_learning_curves.png")

plot_learning_curves()

# %% [markdown]
# ## 图6：综合对比图

# %% 综合对比
def plot_comprehensive_comparison():
    """综合对比：最佳模型 vs 各fraction"""
    if not os.path.exists(FRACTION_CSV):
        print("⚠️ 跳过综合对比")
        return
    
    df = pd.read_csv(FRACTION_CSV)
    if df.empty:
        return
    
    if "error" in df.columns:
        df = df[df["error"].isna()]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # 左图：平均R²趋势
    grouped = df.groupby('fraction')['test_r2_mean'].agg(['mean', 'std', 'min', 'max']).reset_index()
    
    ax1.fill_between(
        grouped['fraction'] * 100,
        grouped['min'],
        grouped['max'],
        alpha=0.2,
        color='blue',
        label='最小-最大范围'
    )
    
    ax1.plot(
        grouped['fraction'] * 100,
        grouped['mean'],
        marker='o',
        markersize=10,
        linewidth=3,
        color='darkblue',
        label='平均R²'
    )
    
    ax1.set_xlabel('训练数据比例 (%)', fontsize=12, weight='bold')
    ax1.set_ylabel('平均R²', fontsize=12, weight='bold')
    ax1.set_title('(A) 总体性能趋势', fontsize=14, weight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)
    
    # 右图：样本效率
    # 计算每个fraction的样本效率（R² per sample）
    if 'n_train' in df.columns:
        efficiency = df.groupby('fraction').apply(
            lambda x: x['test_r2_mean'].mean() / (x['n_train'].mean() + 1)
        ).reset_index()
        efficiency.columns = ['fraction', 'efficiency']
        
        ax2.bar(
            efficiency['fraction'] * 100,
            efficiency['efficiency'],
            width=8,
            color='coral',
            edgecolor='darkred',
            linewidth=2,
            alpha=0.7
        )
        
        ax2.set_xlabel('训练数据比例 (%)', fontsize=12, weight='bold')
        ax2.set_ylabel('样本效率 (R²/样本数)', fontsize=12, weight='bold')
        ax2.set_title('(B) 样本效率分析', fontsize=14, weight='bold')
        ax2.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    out_png = os.path.join(OUT_DIR, "fig6_comprehensive_comparison.png")
    plt.savefig(out_png, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 已输出: fig6_comprehensive_comparison.png")

plot_comprehensive_comparison()

# %% [markdown]
# ## 生成汇总报告

# %% 汇总统计
def generate_summary_report():
    """生成文本汇总报告"""
    report_path = os.path.join(OUT_DIR, "summary_report.txt")
    
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("训练结果汇总报告\n")
        f.write("="*70 + "\n\n")
        
        # 阶段1统计
        if os.path.exists(GRID_CSV):
            df_grid = pd.read_csv(GRID_CSV)
            if not df_grid.empty and "error" in df_grid.columns:
                df_grid = df_grid[df_grid["error"].isna()]
            
            if not df_grid.empty:
                f.write("【阶段1：网格搜索】\n")
                f.write(f"  测试超参组合数: {len(df_grid.groupby(['lr', 'weight_decay']))}\n")
                f.write(f"  总实验次数: {len(df_grid)}\n")
                
                best_row = df_grid.loc[df_grid['test_r2_mean'].idxmax()]
                f.write(f"  最佳超参:\n")
                f.write(f"    - Learning Rate: {best_row['lr']:.0e}\n")
                f.write(f"    - Weight Decay: {best_row['weight_decay']:.0e}\n")
                f.write(f"    - 测试R²: {best_row['test_r2_mean']:.4f}\n")
                f.write(f"    - 验证R²: {best_row['val_r2_best']:.4f}\n\n")
        
        # 阶段2统计
        if os.path.exists(FRACTION_CSV):
            df_frac = pd.read_csv(FRACTION_CSV)
            if not df_frac.empty and "error" in df_frac.columns:
                df_frac = df_frac[df_frac["error"].isna()]
            
            if not df_frac.empty:
                f.write("【阶段2：数据量测试】\n")
                f.write(f"  测试数据比例: {sorted(df_frac['fraction'].unique())}\n")
                f.write(f"  总实验次数: {len(df_frac)}\n\n")
                
                f.write("  各数据比例性能:\n")
                for frac in sorted(df_frac['fraction'].unique()):
                    subset = df_frac[df_frac['fraction'] == frac]
                    mean_r2 = subset['test_r2_mean'].mean()
                    std_r2 = subset['test_r2_mean'].std()
                    f.write(f"    {frac*100:5.1f}%: R²={mean_r2:.4f}±{std_r2:.4f}\n")
                
                f.write("\n  100%数据时各组分R²:\n")
                df_full = df_frac[np.isclose(df_frac['fraction'], 1.0)]
                target_cols = [c for c in df_full.columns 
                              if c.startswith("test_r2_") and c != "test_r2_mean"]
                for col in target_cols:
                    comp_name = col.replace("test_r2_", "")
                    values = df_full[col].dropna()
                    f.write(f"    {comp_name:6s}: {values.mean():.4f}±{values.std():.4f} "
                           f"[{values.min():.4f}, {values.max():.4f}]\n")
        
        f.write("\n" + "="*70 + "\n")
        f.write("报告生成完成\n")
        f.write("="*70 + "\n")
    
    print(f"📄 已输出: summary_report.txt")

generate_summary_report()

# %% 最终总结
print("\n" + "="*70)
print("✅ 所有图表生成完成！")
print("="*70)
print("\n生成的图表:")
print("  1. fig1_grid_search_heatmap.png - 超参网格搜索")
print("  2. fig2_fraction_vs_r2.png - 数据量vs性能")
print("  3. fig3_component_boxplot.png - 组分R²分布")
print("  4. fig4_fold_component_heatmap.png - 折×组分热力图")
print("  5. fig5_component_learning_curves.png - 学习曲线")
print("  6. fig6_comprehensive_comparison.png - 综合对比")
print("  7. summary_report.txt - 文本汇总报告")
print("\n所有文件保存在:", OUT_DIR)

✅ 库导入成功
⚠️ 找不到阶段1结果: F:\model_outputs_2020_resnet_optimized\stage1_grid_search.csv
⚠️ 跳过网格搜索图


C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:142: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.tight_layout()
C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:144: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.savefig(out_png, dpi=300, bbox_inches='tight')


📈 已输出: fig2_fraction_vs_r2.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:223: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.tight_layout()
C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:225: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.savefig(out_png, dpi=300, bbox_inches='tight')


📦 已输出: fig3_component_boxplot.png


...

🔥 已输出: fig4_fold_component_heatmap.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:347: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.tight_layout()
C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:349: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.savefig(out_png, dpi=300, bbox_inches='tight')


📉 已输出: fig5_component_learning_curves.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:425: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.tight_layout()
C:\Users\DELL\AppData\Local\Temp\ipykernel_33896\340059834.py:427: UserWarning: Glyph 178 (\N{SUPERSCRIPT TWO}) missing from current font.
  plt.savefig(out_png, dpi=300, bbox_inches='tight')


📊 已输出: fig6_comprehensive_comparison.png
📄 已输出: summary_report.txt

✅ 所有图表生成完成！

生成的图表:
  1. fig1_grid_search_heatmap.png - 超参网格搜索
  2. fig2_fraction_vs_r2.png - 数据量vs性能
  3. fig3_component_boxplot.png - 组分R²分布
  4. fig4_fold_component_heatmap.png - 折×组分热力图
  5. fig5_component_learning_curves.png - 学习曲线
  6. fig6_comprehensive_comparison.png - 综合对比
  7. summary_report.txt - 文本汇总报告

所有文件保存在: F:\model_outputs_2020_resnet_optimized
